# Output Parser
**Output Parser**는 대규모 언어 모델(LLM, Large Language Model)의 출력 결과를 **애플리케이션에서 활용할 수 있도록 적절한 형식으로 변환**하는 도구이다.
- LLM은 일반적으로 텍스트 형태로 응답을 생성하지만, 이 텍스트는 그대로 활용하기 어려운 경우가 많다.
- Output Parser는 이러한 **비구조적 텍스트 데이터를 구조화된 데이터로 변환**하여 프로그램에서 활용 가능하도록 만든다.
- 예를 들어, 키워드 리스트를 뽑거나 JSON 형식으로 정보를 변환하는 데 사용된다.

## 주요 Output Parser 종류

1. **CommaSeparatedListOutputParser**
   - 쉼표로 구분된 텍스트를 파싱하여 리스트 형태로 변환한다.
   - 예: `"사과, 바나나, 포도"` → `["사과", "바나나", "포도"]`
2. **JsonOutputParser**
   - LLM의 출력이 JSON 형식일 때 이를 Python의 `dict` 객체로 변환한다.
   - JSON(JavaScript Object Notation)은 데이터 구조를 표현하기 위한 경량 포맷이다.
3. **PydanticOutputParser**
   - JSON 데이터를 Python의 [Pydantic](https://docs.pydantic.dev) 모델로 변환한다.
   - Pydantic은 데이터 유효성 검사와 설정 관리에 널리 사용되는 Python 라이브러리이다.
4. **StrOutputParser**
   - 모델의 출력 결과를 단순 문자열로 반환한다.
   - Chat 기반 모델은 Message 객체의 속성으로 LLM 결과를 반환한다. 거기에서 응답 문자열만 추출해서 반환한다.
> `JsonOutputParser`, `PydanticOutputParser` 는 모두 Pydantic을 사용해 데이터 구조(schema)를 정의하고, 해당 구조에 따라 출력을 검증하고 변환한다.

## 주요 메소드
- `parse(text: str)`
  - LLM이 생성한 문자열 응답을 받아 정해진 구조로 변환하여 반환한다.
- `get_format_instructions() -> str`
  - 각 OutputParer가 변환할 수있는 형식으로 LLM이 응답하도록 하는 프롬프트 텍스트를 반환한다.
  - 이 내용을 프롬프트에 넣어서 LLM이 정확한 포맷으로 응답하도록 유도한다.
  
## 참고
- Output Parser는 일반적으로 [`Runnable`](05_chaing_LECL.ipynb#Runnable) 인터페이스를 상속하여 구현되며, `invoke()` 메서드를 통해 실행할 수 있다.
- `invoke()`는 내부적으로 `parse()`를 호출하여 동작한다.
- 필요한 경우 Output Parser를 직접 구현하여 사용자 정의 출력 포맷을 처리할 수도 있다. 


In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## StrOutputParser
- 모델(LLM)의 출력 결과를 string으로 변환하여 반환하는 output parser.
- Chat Model은  Message 객체에서 content 속성값을 추출하여 문자열로 반환한다.

In [2]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 프롬프트 생성
prompt_template = ChatPromptTemplate.from_template(
    "한국의 {topic} 관련된 속담을 {count}개 알려줘."
)
prompt = prompt_template.format(topic="호랑이", count=2)

# LLM 모델 생성
model = ChatOpenAI(model_name="gpt-4o-mini")

# LLM 모델에 prompt를 전달하고 응답 받기.
## prompt -> llm model -> response
response = model.invoke(prompt)

In [3]:
response.response_metadata

{'token_usage': {'completion_tokens': 156,
  'prompt_tokens': 25,
  'total_tokens': 181,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
 'model_name': 'gpt-4o-mini-2024-07-18',
 'system_fingerprint': 'fp_34a54ae93c',
 'id': 'chatcmpl-BhBI64gGL7QxeY7IU1p3HVEDk8FJ9',
 'service_tier': 'default',
 'finish_reason': 'stop',
 'logprobs': None}

In [4]:
print("응답결과:", response.content)

응답결과: 물론입니다! 한국에는 호랑이와 관련된 속담이 여러 가지 있습니다. 두 가지를 소개해 드릴게요.

1. **호랑이 굴에 가야 호랑이 새끼를 잡는다.**  
   - 이 속담은 큰 성취를 원한다면 위험을 감수해야 한다는 의미입니다. 

2. **호랑이 뒷다리 잡아라.**  
   - 이 속담은 적극적으로 행동하여 기회를 잡으라는 의미로, 망설이지 말고 행동하라는 교훈을 담고 있습니다.

이 두 가지 속담은 호랑이가 지닌 강력한 이미지와 관련하여 인간의 삶에 대한 교훈을 전달하고 있습니다.


In [5]:
parser = StrOutputParser()  # Message 객체에서 content 속성(메세지)의 값만 추출.
res = parser.invoke(response) # LLM 모델의 응답결과
print(res)

물론입니다! 한국에는 호랑이와 관련된 속담이 여러 가지 있습니다. 두 가지를 소개해 드릴게요.

1. **호랑이 굴에 가야 호랑이 새끼를 잡는다.**  
   - 이 속담은 큰 성취를 원한다면 위험을 감수해야 한다는 의미입니다. 

2. **호랑이 뒷다리 잡아라.**  
   - 이 속담은 적극적으로 행동하여 기회를 잡으라는 의미로, 망설이지 말고 행동하라는 교훈을 담고 있습니다.

이 두 가지 속담은 호랑이가 지닌 강력한 이미지와 관련하여 인간의 삶에 대한 교훈을 전달하고 있습니다.


In [6]:
# prompt_template -> model -> output parser
chain = prompt_template | model | parser

res = chain.invoke({"topic":"사람의 정신력", "count":3})
print(res)

한국에는 사람의 정신력이나 의지를 강조하는 여러 속담이 있습니다. 그 중에서 몇 가지를 소개하겠습니다:

1. **밑 빠진 독에 물 붓기**  
   - 어떤 일을 해도 성과가 없거나, 아무리 노력해도 보람이 없는 상황을 의미합니다. 하지만 이 속담은 끈질기게 노력하는 것의 중요성을 되새깁니다.

2. **어제의 실패는 오늘의 교훈이다**  
   - 실패를 겪더라도 그로부터 배우고 성장할 수 있다는 의미로, 긍정적인 마음가짐과 정신력을 강조합니다.

3. **꾸준함이 이루는 것**  
   - 지속적인 노력과 인내가 결국 성과를 이룬다는 의미입니다. 꾸준한 정신력이 중요하다는 메시지를 담고 있습니다.

이러한 속담들은 어려운 상황에서도 포기하지 않고 끝까지 버티는 정신력을 강조합니다.


## CommaSeparatedListOutputParser

- 쉼표로 구분된 텍스트를 파싱하여 리스트 형태로 변환한다.
  - "a,b,c" => ['a','b','c']

In [7]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

parser = CommaSeparatedListOutputParser()
res_txt = "사과,배,귤,수박,오렌지"
print(parser.invoke(res_txt))

['사과', '배', '귤', '수박', '오렌지']


In [8]:
# 출력 형식을 지정하는 프롬프트를 조회
format_string = parser.get_format_instructions()
print(format_string)


Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [9]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

from textwrap import dedent
prompt_template = ChatPromptTemplate.from_template(
    dedent("""
    # instruction
    {subject}의 이름 다섯개를 나열해주세요.
    
    # output indicator
    {format_instructions}
    """),
    partial_variables={"format_instructions": parser.get_format_instructions()}
)
# partial_variables={변수명:넣을값,..} : 템플릿의 placeholder 변수에 넣을 값을 
#   PromptTemplate객체를 생성할 때 넣겠다. placeholder에 넣을 값이 있는데 함수나 메소드호출을 통해 
#   그 값을 가져와야 하는 경우 사용.

model = ChatOpenAI(model_name="gpt-4o-mini")

# prompt 생성
prompt = prompt_template.invoke({"subject":"동물"})
# LLM에 요청
response = model.invoke(prompt)

In [10]:
# 응답 확인
print(response.content)
# parser를 이용해 List로 변환.
res = parser.invoke(response)
print(type(res))
print(res)

개, 고양이, 코끼리, 사자, 기린
<class 'list'>
['개', '고양이', '코끼리', '사자', '기린']


In [11]:
res[0]

'개'

In [12]:
# chain으로 구성
chain = prompt_template | model | parser

res = chain.invoke({"subject":"한국산 자동차"})
res

['소나타', '스포티지', '모하비', '쏘렌토', '그랜저']

## JsonOutputParser

- JSON 형식의 응답을 dictionary로 반환한다.
- JSON 형식을 정하려는 경우 [Pydantic](Ref_typing_Pydantic.ipynb)을 이용해 JSON 스키마를 정의하여 JsonOutputParser 생성시 전달한다.
  - Pydantic 모델클래스를 이용해 LLM 모델이 응답할 때 json의 어떤 key에 어떤 응답을 작성할 지 Field로 정의한다.
  - Schema 지정은 필수는 아니다. 
- LLM이 JSON Schema를 따르는 형태로 응답을 하면 JsonOutputParser는 Dictionary로 변환한다.

In [13]:
from langchain_core.output_parsers import JsonOutputParser

parser = JsonOutputParser()
res_text = """
{
    "name":"홍길동",
    "age": 20,
    "address": "서울",
    "hobby": ["독서", "게임"]
}
"""
res_dict = parser.invoke(res_text)
print(type(res_dict))
res_dict
res_dict["name"], res_dict["address"]

<class 'dict'>


('홍길동', '서울')

In [14]:
parser.get_format_instructions()

'Return a JSON object.'

In [15]:
prompt_template = ChatPromptTemplate.from_template(
    "{name}에 대해서 설명해줘.\n{format_instructions}",
    partial_variables={"format_instructions":parser.get_format_instructions()}
)
model = ChatOpenAI(model_name="gpt-4o-mini")

prompt = prompt_template.invoke({"name":"제네시스"})
res = model.invoke(prompt)

In [16]:
print(res.content)

```json
{
  "브랜드": "제네시스",
  "설명": "제네시스는 현대자동차 그룹의 고급 차량 브랜드입니다. 2015년에 독립 브랜드로 출범하였으며, 고급 승용차와 SUV를 제조합니다. 제네시스는 고급스러운 디자인, 첨단 기술, 편안한 승차감 등을 특징으로 하며, 글로벌 시장에서도 인지도를 높이고 있습니다.",
  "모델": [
    {
      "이름": "G70",
      "설명": "스포츠 세단으로, 역동적인 디자인과 우수한 주행 성능을 강조"
    },
    {
      "이름": "G80",
      "설명": "고급 세단으로, 럭셔리한 내장과 최신 기술 장비를 갖춤"
    },
    {
      "이름": "G90",
      "설명": "플래그십 세단으로, 정숙성, 안락함, 최신 안전 기술이 특징"
    },
    {
      "이름": "GV70",
      "설명": "중형 럭셔리 SUV로, 실용성과 고급스러움을 모두 갖춘 모델"
    },
    {
      "이름": "GV80",
      "설명": "대형 럭셔리 SUV로, 넓은 공간과 다양한 편의 장치를 제공"
    }
  ],
  "미션": "고객에게 최고의 경험을 제공하고 지속 가능한 미래를 위한 혁신적 기술 개발을 추구"
}
```


In [17]:
res_dict = parser.invoke(res)
print(type(res.content), type(res_dict))

<class 'str'> <class 'dict'>


In [18]:
res_dict["아이폰"]['첫 번째 출시']

KeyError: '아이폰'

In [ ]:
# 출력 스키마를 정의
## json 형식을 설계.
from pydantic import BaseModel, Field

class ItemSchema(BaseModel):
    # JSON에 포함될 항목들을 class변수로 정의. 변수명: 타입 = Field(설명)
    name: str = Field(description="제품의 이름")
    info: str = Field(description="제품에 대한 정보")
    release_date: str = Field(description="제품이 출시된 일시. yyyy-mm-dd 형식")
    price: int = Field(description="제품의 한국 가격.")

parser = JsonOutputParser(pydantic_object=ItemSchema)
# print(parser.get_format_instructions())

prompt_template = ChatPromptTemplate.from_template(
    dedent("""
    # instruction
    {name}에 대해서 설명해주세요.
           
    # output indicator
    {format_instructions}
    """),
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

model = ChatOpenAI(model_name="gpt-4o-mini")

In [ ]:
p = prompt_template.invoke({"name":"Galaxy S24"})
# print(p.messages[0].content)
res = model.invoke(p)
# print(res.content)
response = parser.invoke(res)

In [ ]:
response

{'name': 'Galaxy S24',
 'info': 'Galaxy S24는 최신 모바일 기술이 집약된 스마트폰으로, 뛰어난 성능과 카메라 기능을 제공합니다. 진보된 디스플레이와 강력한 프로세서, 개선된 배터리 수명이 특징입니다.',
 'release_date': '2024-02-01',
 'price': 1099000}

## PydanticOutputParser

- JSON 형태로 받은 응답을 Pydantic 모델로 변환하여 반환한다.
- 구현은 JsonOutputParser와 동일한데 parsing 결과를 pydantic 모델타입으로 반환한다.

In [ ]:
from langchain_core.output_parsers import PydanticOutputParser

parser = PydanticOutputParser(pydantic_object=ItemSchema)
# JsonOutputParser와 동일한 format instruction을 생성.
## 응답: JSON ->  Parser: Pydatic Model객체로 변환.
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"description": "제품의 이름", "title": "Name", "type": "string"}, "info": {"description": "제품에 대한 정보", "title": "Info", "type": "string"}, "release_date": {"description": "제품이 출시된 일시. yyyy-mm-dd 형식", "title": "Release Date", "type": "string"}, "price": {"description": "제품의 한국 가격.", "title": "Price", "type": "integer"}}, "required": ["name", "info", "release_date", "price"]}
```


In [ ]:
prompt_template = ChatPromptTemplate.from_template(
    dedent("""
    # instruction
    {name}에 대해서 설명해주세요.
           
    # output indicator
    {format_instructions}
    """),
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

model = ChatOpenAI(model_name="gpt-4o-mini")

In [ ]:
prompt = prompt_template.invoke({"name":"Mac Book"})
res = model.invoke(prompt)
response = parser.invoke(res)

In [ ]:
print(type(response))

<class '__main__.ItemSchema'>


In [ ]:
print("제품이름:", response.name)
print("정보:", response.info)
print(response.release_date, response.price)

제품이름: MacBook
정보: 애플이 제조하는 노트북 컴퓨터로, 고급스러운 디자인과 강력한 성능을 자랑합니다. macOS 운영 체제를 사용하며, 다양한 모델이 존재합니다.
2021-10-18 1599000


In [ ]:
chain = prompt_template | model | parser

response = chain.invoke({"name":"아이폰"})

In [ ]:
response

ItemSchema(name='아이폰', info='아이폰은 애플(Apple)에서 개발한 스마트폰으로, 매력적인 디자인과 사용자 친화적인 인터페이스, 강력한 성능을 자랑합니다. 다양한 애플리케이션과 서비스를 제공하며, 전 세계적으로 많은 사용자층을 보유하고 있습니다.', release_date='2023-09-12', price=1250000)